# TM10007 Assignment template -- ECG data

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [1]:
import zipfile
import os
import pandas as pd

with zipfile.ZipFile('./ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

data = pd.read_csv('./ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

FileNotFoundError: [Errno 2] No such file or directory: './ecg/ecg_data.zip'